In [1]:
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

import IO
import torch
from sequence_dataset import tensor_imshow, tensor_depthshow
import cv2
import inverse_warp

In [2]:
disp, scale = IO.readPFM("/home/ai/Data/FlyingThings3D_subset/train/disparity/left/0000000.pfm")
image = IO.readImage("/home/ai/Data/FlyingThings3D_subset/train/image_clean/left/0000000.png")
disp = torch.from_numpy(np.ascontiguousarray(disp)).unsqueeze(0)
image = torch.from_numpy(np.ascontiguousarray(image.transpose(2, 0, 1))).unsqueeze(0) / 255.0

In [3]:
f = 1050.0
K = torch.tensor([
    [f,   0.0, 479.5],
    [0.0,   f, 269.5],
    [0.0, 0.0,   1.0]
]).unsqueeze(0)
Kinv = K.inverse()
pixels = torch.tensor([
    [[0.0, 479.5, 959, 479.5],[0.0, 269.5, 539.5, 0.0],[1.0, 1.0, 1.0, 1.0]]
])
rays = Kinv @ pixels

print(Kinv.shape)
print(pixels.shape)
print(rays.shape)

print(pixels)
print(rays)
print(rays[0,0])
print(rays[0,1])
print(rays[0,2])

torch.Size([1, 3, 3])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
tensor([[[  0.0000, 479.5000, 959.0000, 479.5000],
         [  0.0000, 269.5000, 539.5000,   0.0000],
         [  1.0000,   1.0000,   1.0000,   1.0000]]])
tensor([[[-0.4567,  0.0000,  0.4567,  0.0000],
         [-0.2567,  0.0000,  0.2571, -0.2567],
         [ 1.0000,  1.0000,  1.0000,  1.0000]]])
tensor([-0.4567,  0.0000,  0.4567,  0.0000])
tensor([-0.2567,  0.0000,  0.2571, -0.2567])
tensor([1., 1., 1., 1.])


In [33]:



pose = torch.tensor([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]).unsqueeze(0)
#depth = torch.ones((image.shape[-2], image.shape[-1])).type_as(image).unsqueeze(0) * 2
depth = (f * 1.0 / -disp).clamp(max=30)

warped, valid, pixels, rays, world = inverse_warp.inverse_warp(image, depth, pose, K)
D = depth.unsqueeze(1)
world2 = D * rays

print(D.shape)

print("---")
print(pixels[0,:,0,0])
print(rays[0,:,0,0])
print(depth[0,0,0])
print(world[0,:,0,0])
print(world2[0,:,0,0])
print("---")
print(pixels[0,:,-1,-1])
print(rays[0,:,-1,-1])
print(depth[0,-1,-1])
print(world[0,:,-1,-1])
print(world2[0,:,-1,-1])

torch.Size([1, 1, 540, 960])
---
tensor([0., 0., 1.])
tensor([-0.4567, -0.2567,  1.0000])
tensor(30.)
tensor([-13.7000,  -7.7000,  30.0000,   1.0000])
tensor([-13.7000,  -7.7000,  30.0000])
---
tensor([959., 539.,   1.])
tensor([0.4567, 0.2567, 1.0000])
tensor(23.9487)
tensor([10.9366,  6.1468, 23.9487,  1.0000])
tensor([10.9366,  6.1468, 23.9487])


In [29]:
fig = plt.figure()
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax1.scatter(rays[0,0,...], rays[0,1,...])
ax2.scatter(world[0,0,...], world[0,1,...])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
fig = plt.figure()
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')
ax1.scatter(rays[0,0,...][::1], rays[0,1,...][::1], rays[0,2,...][::1])
ax2.scatter(world[0,0,...][::1], world[0,1,...][::1], world[0,2,...][::1])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …